This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

In [1]:
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190726104336-0001
KERNEL_ID = aba63720-5130-4357-851c-328e47e0f22d
--2019-07-26 10:43:38--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet [following]
--2019-07-26 10:43:39--  https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.68.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.68.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59032 (58K) [application/octet-stream]
Saving to: 'a2.parquet.1'

a2.parquet.1        100%[===================>]  57.65K   253KB/s    in 0.2s  

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [2]:
df = spark.read.load('a2.parquet')

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

Create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [3]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=['X', 'Y', 'Z'], outputCol='features')

Instantiate a classifier. For the sake of simplicity, I'll choose LR (should've normalised the features first, but the classification results don't matter in this assignment.)

In [4]:
from pyspark.ml.classification import LogisticRegression
classifier = LogisticRegression(
    featuresCol='features',
    predictionCol='prediction', 
    labelCol='CLASS',
    maxIter=10,
    regParam=0.01
)

Let’s train and evaluate…


In [5]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [6]:
model = pipeline.fit(df)

In [7]:
prediction = model.transform(df)

In [8]:
prediction.show()

+-----+-----------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|            features|       rawPrediction|         probability|prediction|
+-----+-----------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|[380.664340054951...|[0.13824737692205...|[0.53450690276559...|       0.0|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|[104.743242992096...|[0.13870067495190...|[0.53461968575308...|       0.0|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|[118.114692361299...|[0.13993770076481...|[0.53492744638363...|       0.0|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.186625139515

In [9]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction) 

0.534516765285996

## Submission

In [10]:
!rm -Rf a2_m2.json

In [11]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [12]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2019-07-26 10:44:31--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.68.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.68.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: 'rklib.py'

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2019-07-26 10:44:32 (52.1 MB/s) - 'rklib.py' saved [2540/2540]



In [13]:
import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('a2_m2.json.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('a2_m2.json', zipf)
zipf.close()

In [14]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [16]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = 'xquy.vu@gmail.com'
secret = 'RwMvrfqNnF5lo5VS'

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~thUOQa-SEemN2g6-65EF-g","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
